In [1]:
!pip install requests requests_cache newsapi-python fuzzywuzzy python-Levenshtein pandas matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 37.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import requests
from newsapi import NewsApiClient
from fuzzywuzzy import fuzz
from datetime import datetime
import matplotlib.pyplot as plt

In [3]:
import requests_cache
requests_cache.install_cache('api_cache', expire_after=3600)

In [4]:
df = pd.read_csv('/content/Forbes_2000_top_company_CLNQ11.csv')
df.head()

,2022 Ranking,Organization Name,Industry,Country,Year Founded,CEO,Revenue (Billions),Profits (Billions),Assets (Billions),Market Value (Billions),Total Employees
0,1,Berkshire Hathaway,Diversified Financials,United States,1939,Warren Edward Buffett,276.09,89.80,958.78,741.48,372000.0
1,2,ICBC,Banking,China,1984,Shu Gu,208.13,54.03,5518.51,214.42,449296
2,3,Saudi Arabian Oil Company (Saudi Aramco),Oil & Gas Operations,Saudi Arabia,1933,Amin bin Hasan Al-Nasser,400.38,105.36,576.04,2292.08,68493.0
3,4,JPMorgan Chase,Diversified Financials,United States,2000,Jamie Dimon,124.54,42.12,3954.69,374.45,271025.0
4,5,China Construction Bank,Banking,China,2014,Wang Zuji,202.07,46.89,4746.95,181.32,352621


In [5]:
df = df.head(10).reset_index(drop=True)

In [6]:
df.head(60)

,2022 Ranking,Organization Name,Industry,Country,Year Founded,CEO,Revenue (Billions),Profits (Billions),Assets (Billions),Market Value (Billions),Total Employees
0,1,Berkshire Hathaway,Diversified Financials,United States,1939,Warren Edward Buffett,276.09,89.80,958.78,741.48,372000.0
1,2,ICBC,Banking,China,1984,Shu Gu,208.13,54.03,5518.51,214.42,449296
2,3,Saudi Arabian Oil Company (Saudi Aramco),Oil & Gas Operations,Saudi Arabia,1933,Amin bin Hasan Al-Nasser,400.38,105.36,576.04,2292.08,68493.0
3,4,JPMorgan Chase,Diversified Financials,United States,2000,Jamie Dimon,124.54,42.12,3954.69,374.45,271025.0
4,5,China Construction Bank,Banking,China,2014,Wang Zuji,202.07,46.89,4746.95,181.32,352621
5,6,Amazon,Retailing,United States,1994,Andy Jassy,469.82,33.36,420.55,1468.40,1608000.0
6,7,Apple,Technology Hardware & Equipment,United States,1976,Tim Cook,378.70,100.56,381.19,2640.32,154000.0
7,8,Agricultural Bank of China,Banking,China,1979,Gu Shu,181.42,37.38,4561.05,133.38,455174.0
8,9,Bank of America,Banking,United States,1998,Brian Moynihan,96.83,31.00,3238.22,303.10,208000.0
9,10,Toyota Motor,Consumer Durables,Japan,1937,Akio Toyoda,281.75,28.15,552.46,237.73,366283.0


In [7]:
# ============================================================
# DISPLAY DATA FOR A SPECIFIC INDUSTRY
# ============================================================

# Choose the industry you want to view
industry_name = "Banking"  # You can change this to any other industry name

# Filter dataset for the selected industry
industry_df = df[df['Industry'] == industry_name].copy()

# Display the industry-specific data
print(f"--- Displaying data for industry: {industry_name} ---")
display(industry_df)

--- Displaying data for industry: Banking ---


,2022 Ranking,Organization Name,Industry,Country,Year Founded,CEO,Revenue (Billions),Profits (Billions),Assets (Billions),Market Value (Billions),Total Employees
1,2,ICBC,Banking,China,1984,Shu Gu,208.13,54.03,5518.51,214.42,449296
4,5,China Construction Bank,Banking,China,2014,Wang Zuji,202.07,46.89,4746.95,181.32,352621
7,8,Agricultural Bank of China,Banking,China,1979,Gu Shu,181.42,37.38,4561.05,133.38,455174.0
8,9,Bank of America,Banking,United States,1998,Brian Moynihan,96.83,31.00,3238.22,303.10,208000.0


In [8]:
pip install google-api-python-client

In [9]:
import os
from google.colab import userdata
from googleapiclient.discovery import build

def find_official_website(company_name):
    """Searches Google to find the official website of a company using Colab Secrets."""

    # --- Access keys from Colab's Secrets manager ---
    try:
        api_key = userdata.get("GOOGLE_API_KEY")
        cse_id = userdata.get("GOOGLE_CSE_ID")
    except userdata.SecretNotFoundError:
        return "Error: Secrets not found. Please add GOOGLE_API_KEY and GOOGLE_CSE_ID to Colab's Secrets manager."

    if not company_name:
        return "Error: Missing company name."

    try:
        service = build("customsearch", "v1", developerKey=api_key)
        query = f"official website of {company_name}"
        res = service.cse().list(q=query, cx=cse_id, num=1).execute()

        if items := res.get('items'):
            return items[0].get('link')
        return "Could not find a website for the company."

    except Exception as e:
        return f"An error occurred: {e}"

# --- Example Usage in Colab ---
company = "Microsoft"
website = find_official_website(company)
print(f"The website for '{company}' is likely: {website}")

The website for 'Microsoft' is likely: https://www.microsoft.com/


In [10]:
from tqdm.auto import tqdm
tqdm.pandas()

df['Company Website'] = df['Organization Name'].progress_apply(find_official_website)

print("\nProcessing complete!")

# Display the first 10 rows of the updated DataFrame
display(df.head(10))

  0%|          | 0/10 [00:00<?, ?it/s]


Processing complete!


,2022 Ranking,Organization Name,Industry,Country,Year Founded,CEO,Revenue (Billions),Profits (Billions),Assets (Billions),Market Value (Billions),Total Employees,Company Website
0,1,Berkshire Hathaway,Diversified Financials,United States,1939,Warren Edward Buffett,276.09,89.80,958.78,741.48,372000.0,https://www.berkshirehathaway.com/
1,2,ICBC,Banking,China,1984,Shu Gu,208.13,54.03,5518.51,214.42,449296,https://www.icbc.com.cn/icbc/sy/
2,3,Saudi Arabian Oil Company (Saudi Aramco),Oil & Gas Operations,Saudi Arabia,1933,Amin bin Hasan Al-Nasser,400.38,105.36,576.04,2292.08,68493.0,https://www.aramco.com/en
3,4,JPMorgan Chase,Diversified Financials,United States,2000,Jamie Dimon,124.54,42.12,3954.69,374.45,271025.0,https://www.chase.com/
4,5,China Construction Bank,Banking,China,2014,Wang Zuji,202.07,46.89,4746.95,181.32,352621,https://www.ccb.com/en/home/index.html
5,6,Amazon,Retailing,United States,1994,Andy Jassy,469.82,33.36,420.55,1468.40,1608000.0,https://www.amazon.com/
6,7,Apple,Technology Hardware & Equipment,United States,1976,Tim Cook,378.70,100.56,381.19,2640.32,154000.0,https://www.apple.com/
7,8,Agricultural Bank of China,Banking,China,1979,Gu Shu,181.42,37.38,4561.05,133.38,455174.0,https://www.abchina.com/en/
8,9,Bank of America,Banking,United States,1998,Brian Moynihan,96.83,31.00,3238.22,303.10,208000.0,https://www.bankofamerica.com/
9,10,Toyota Motor,Consumer Durables,Japan,1937,Akio Toyoda,281.75,28.15,552.46,237.73,366283.0,https://global.toyota/en/index.html


In [11]:
# ============================================================
# INDUSTRY-SPECIFIC PCA SCORING (For One Target Industry)
# ============================================================

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np

# --- Define your target industry here ---
TARGET_INDUSTRY = 'Banking'  # Change this to any industry name

# --- 1. PCA Scoring Function for a Single Industry ---
def calculate_pca_score(df):
    """
    Calculates PCA score, normalized PCA score, and ranking within the selected industry.
    """
    metrics = [
        'Revenue (Billions)', 'Profits (Billions)', 'Assets (Billions)',
        'Market Value (Billions)', 'Total Employees'
    ]

    # Drop rows with missing values in key metrics
    df_cleaned = df.dropna(subset=metrics).copy()

    # PCA requires at least 2 rows
    if len(df_cleaned) < 2:
        print("Not enough data for PCA in this industry.")
        return df

    # Step 1: Standardize data
    X_scaled = StandardScaler().fit_transform(df_cleaned[metrics])

    # Step 2: Apply PCA
    pca = PCA(n_components=1)
    df_cleaned['PCA Score'] = pca.fit_transform(X_scaled)

    # Step 3: Normalize PCA Score to [0, 1] range
    scaler = MinMaxScaler()
    df_cleaned['PCA ScoreN'] = scaler.fit_transform(df_cleaned[['PCA Score']])

    # Step 4: Rank within industry
    df_cleaned = df_cleaned.sort_values(by='PCA Score', ascending=False).reset_index(drop=True)
    df_cleaned['Rank_in_Industry'] = df_cleaned.index + 1

    # Step 5: Extract PCA component weights (for reference)
    weights_df = pd.DataFrame({
        'Metric': metrics,
        'Learned Weight': pca.components_[0]
    }).sort_values(by='Learned Weight', ascending=False).reset_index(drop=True)

    return df_cleaned, weights_df


# --- 2. Run PCA Only for the Target Industry ---
def run_single_industry_analysis(full_df, target_industry):
    """
    Runs PCA and ranking for only the target industry.
    """
    if 'Industry' not in full_df.columns:
        raise ValueError("DataFrame must contain an 'Industry' column.")

    industry_df = full_df[full_df['Industry'] == target_industry].copy()

    if industry_df.empty:
        print(f"No data found for industry: {target_industry}")
        return None, None

    print(f"Running PCA analysis for industry: {target_industry}")
    scored_df, weights_df = calculate_pca_score(industry_df)
    return scored_df, weights_df


# ============================================================
# RUN THE ANALYSIS (Example)
# ============================================================

# Example: assuming your main DataFrame is named 'df'
# df = pd.read_csv("your_dataset.csv")  # Uncomment if reading from file

final_results_df, learned_weights_df = run_single_industry_analysis(df, TARGET_INDUSTRY)

# Display results
if final_results_df is not None:
    print("\n--- PCA Results for Target Industry ---")
    display(final_results_df[['Industry', 'Rank_in_Industry', 'Organization Name', 'PCA ScoreN']])

    print("\n--- Learned Weights (PCA Loadings) ---")
    display(learned_weights_df)

    # Select only the new columns you want to add to the original dataset
    pca_cols_to_merge = ['Organization Name', 'Rank_in_Industry', 'PCA ScoreN']

    # Merge the new columns with the original dataset based on 'Organization Name'
    df = industry_df.merge(final_results_df[pca_cols_to_merge], on='Organization Name', how='left')

    print("\n--- Updated Original Dataset with PCA Columns ---")
    # The full, original DataFrame is now shown with the new columns added.
    # Note that companies not in the 'Banking' industry have NaN (blank) values.
    display(df)

Running PCA analysis for industry: Banking

--- PCA Results for Target Industry ---


,Industry,Rank_in_Industry,Organization Name,PCA ScoreN
0,Banking,1,ICBC,1.000000
1,Banking,2,China Construction Bank,0.793350
2,Banking,3,Agricultural Bank of China,0.790166
3,Banking,4,Bank of America,0.000000



--- Learned Weights (PCA Loadings) ---


,Metric,Learned Weight
0,Revenue (Billions),0.485869
1,Assets (Billions),0.479993
2,Total Employees,0.459323
3,Profits (Billions),0.418086
4,Market Value (Billions),-0.384400



--- Updated Original Dataset with PCA Columns ---


,2022 Ranking,Organization Name,Industry,Country,Year Founded,CEO,Revenue (Billions),Profits (Billions),Assets (Billions),Market Value (Billions),Total Employees,Rank_in_Industry,PCA ScoreN
0,2,ICBC,Banking,China,1984,Shu Gu,208.13,54.03,5518.51,214.42,449296,1,1.000000
1,5,China Construction Bank,Banking,China,2014,Wang Zuji,202.07,46.89,4746.95,181.32,352621,2,0.793350
2,8,Agricultural Bank of China,Banking,China,1979,Gu Shu,181.42,37.38,4561.05,133.38,455174.0,3,0.790166
3,9,Bank of America,Banking,United States,1998,Brian Moynihan,96.83,31.00,3238.22,303.10,208000.0,4,0.000000


In [12]:
# ---------------------------
# 0) Install Libraries
# ---------------------------
!pip install wikipedia requests pandas transformers torch --quiet

import pandas as pd
import wikipedia
from transformers import pipeline

# ---------------------------
# 1) Load the Pre-Trained Zero-Shot Model
# ---------------------------
try:
    classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    print("✅ Zero-shot classification model loaded successfully.")
except Exception as e:
    print(f"Error loading model: {e}")
    classifier = None

# ---------------------------
# 2) Comprehensive, Tiered Technology Lists
# ---------------------------
core_tech = [
    "Cloud Computing", "AWS", "Microsoft Azure", "Google Cloud Platform", "Cybersecurity", "CRM", "Salesforce",
    "ERP", "SAP", "Oracle", "Microsoft 365", "Google Workspace", "Data Analytics", "Business Intelligence",
    "Machine Learning", "Artificial Intelligence", "Python", "SQL", "Linux", "Windows Server"
]
industry_specific_tech = {
    "Finance": [
        "FinTech", "Blockchain", "High-Frequency Trading", "Algorithmic Trading", "Payment Gateway", "Stripe",
        "Core Banking Systems", "Risk Management", "Compliance Tech", "KYC", "AML", "SWIFT"
    ],
    "Healthcare": [
        "Electronic Health Record (EHR)", "Epic Systems", "Cerner", "Telemedicine", "Medical Imaging", "DICOM",
        "HL7", "FHIR", "Bioinformatics", "Genomics", "CRISPR", "Robotic Surgery", "HIPAA Compliance"
    ],
    "IT & Software": [
        "SaaS", "Microservices", "API", "Kubernetes", "Docker", "Terraform", "Ansible", "CI/CD", "Jenkins",
        "JavaScript", "React", "Node.js", "Java", "Spring Boot", "DevOps", "Agile Methodology"
    ],
    "Retailing": [
        "E-commerce", "Shopify", "Magento", "Supply Chain Management", "Point of Sale (POS)", "Inventory Management",
        "Personalization Engine", "Recommendation Engine", "Last-Mile Delivery", "Omnichannel Retail"
    ],
    "Oil & Gas": [
        "SCADA", "GIS", "Reservoir Simulation", "Drilling Technology",
        "Pipeline Management", "LNG", "Carbon Capture", "Renewable Energy"
    ],
    "Automotive": [
        "Internet of Things (IoT)", "Electric Vehicles (EV)", "Autonomous Driving", "LiDAR", "ADAS",
        "Battery Technology", "Just-In-Time (JIT) Manufacturing", "CAD", "PLM"
    ]
}

# ---------------------------
# 3) Industry Mapping
# ---------------------------
# Maps the specific industry names from your CSV to our general tech categories
industry_map = {
    'Diversified Financials': 'Finance', 'Banking': 'Finance', 'Insurance': 'Finance',
    'Oil & Gas Operations': 'Oil & Gas', 'Utilities': 'Oil & Gas',
    'IT Software & Services': 'IT & Software', 'Technology Hardware & Equipment': 'IT & Software', 'Semiconductors': 'IT & Software',
    'Retailing': 'Retailing', 'Hotels, Restaurants & Leisure': 'Retailing',
    'Automotive': 'Automotive', 'Consumer Durables': 'Automotive',
    'Healthcare': 'Healthcare', 'Drugs & Biotechnology': 'Healthcare',
    'Food, Drink & Tobacco': 'Retailing'
    # Add more mappings here as needed
}

# ---------------------------
# 4) Analysis and Helper Functions
# ---------------------------
def fetch_wikipedia_summary(company_name):
    try:
        page = wikipedia.page(wikipedia.search(company_name, results=1)[0], auto_suggest=False)
        return page.summary
    except Exception: return ""

def aggregate_company_text(company_name):
    return fetch_wikipedia_summary(company_name)

def get_technographics_with_zeroshot(text, tech_list):
    if not classifier or not text or not tech_list: return {}, [], 0.0
    try:
        results = classifier(text, tech_list, multi_label=True)
        all_scores = {label: round(score, 3) for label, score in zip(results['labels'], results['scores'])}
        detected = [tech for tech, score in all_scores.items() if score >= 0.60]
        tech_score = sum(all_scores.values()) / len(all_scores) if all_scores else 0.0
        return all_scores, detected, round(tech_score, 3)
    except Exception as e:
        print(f"  -> Error during classification: {e}")
        return {}, [], 0.0

# ---------------------------
# 5) Main Enrichment Function
# ---------------------------
def run_industry_specific_enrichment(df, tech_category):
    results = []

    if tech_category:
        tech_list_for_analysis = core_tech + industry_specific_tech.get(tech_category, [])
    else:
        tech_list_for_analysis = core_tech

    for _, row in df.iterrows():
        cname = row['Organization Name']
        print(f"Processing: {cname}...")

        agg_text = aggregate_company_text(cname)
        if not agg_text:
            print(f"  -> Skipping {cname}, no Wikipedia summary found.")
            results.append({'Organization Name': cname, 'tech_score_model': 0, 'detected_techs_model': []})
            continue

        _, detected, tscore = get_technographics_with_zeroshot(agg_text, tech_list_for_analysis)
        results.append({'Organization Name': cname, 'tech_score_model': tscore, 'detected_techs_model': detected})
        print(f"  -> Done. Score: {tscore}")

    results_df = pd.DataFrame(results)
    enriched_df = pd.merge(df, results_df, on='Organization Name', how='left')

    enriched_df['Tech_Percentile_in_Domain'] = enriched_df['tech_score_model'].rank(pct=True)

    return enriched_df




    # ---------------------------
# 5) Load Data and Run Focused Analysis
# ---------------------------
try:
    #

    # STEP 1: Get and display all unique industries for the user to choose from
    available_industries = sorted(df['Industry'].unique())
    print("--- Available Industries for Analysis ---")
    for industry in available_industries:
        print(f"- {industry}")
    print("-" * 40)

    # <<< --- STEP 2: SET YOUR TARGET INDUSTRY HERE --- >>>
    TARGET_INDUSTRY = 'Banking'

    # STEP 3: Filter the DataFrame to get all companies in the selected domain
    peer_group_df = df[df['Industry'] == TARGET_INDUSTRY].copy()

    if peer_group_df.empty:
        print(f"\nERROR: Industry '{TARGET_INDUSTRY}' not found or has no companies. Please choose from the list above.")
    else:
        print(f"\nAnalyzing {len(peer_group_df)} companies in the '{TARGET_INDUSTRY}' domain...")

        # STEP 4: Run the enrichment only on this peer group
        tech_category_for_analysis = industry_map.get(TARGET_INDUSTRY)
        df_final = run_industry_specific_enrichment(peer_group_df, tech_category_for_analysis)

        # STEP 5: Display the final competitive ranking
        print("\n\n--- Competitive Analysis Complete ---")
        print(f"The table below shows the technology leaders in the '{TARGET_INDUSTRY}' domain.")

        df_sorted = df_final.sort_values(by='Tech_Percentile_in_Domain', ascending=False)

        display_cols = ['Organization Name', 'tech_score_model', 'Tech_Percentile_in_Domain']

        styled_df = df_sorted[display_cols].style.format({
            'tech_score_model': '{:.3f}',
            'Tech_Percentile_in_Domain': '{:.2%}'
        }).background_gradient(cmap='Greens', subset=['Tech_Percentile_in_Domain'])

        display(styled_df)



        tech_cols_to_merge = ['Organization Name', 'tech_score_model']
        df=industry_df.merge(df_final[tech_cols_to_merge], on='Organization Name', how='left')

        print("\n\n--- Original Dataset Updated with tech_score_model ---")
        display(df.head())  # show first 10 for quick check

except FileNotFoundError:
    print("\nERROR: The file 'Forbes_2000_top_company_CLNQ11.csv' was not found. Please make sure it's uploaded.")
except Exception as e:
    print(f"\nAn error occurred: {e}")

  Preparing metadata (setup.py) ... done


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

...

Device set to use cpu


✅ Zero-shot classification model loaded successfully.
--- Available Industries for Analysis ---
- Banking
----------------------------------------

Analyzing 4 companies in the 'Banking' domain...
Processing: ICBC...
  -> Done. Score: 0.267
Processing: China Construction Bank...
  -> Done. Score: 0.257
Processing: Agricultural Bank of China...
  -> Done. Score: 0.241
Processing: Bank of America...
  -> Done. Score: 0.192


--- Competitive Analysis Complete ---
The table below shows the technology leaders in the 'Banking' domain.


,Organization Name,tech_score_model,Tech_Percentile_in_Domain
0,ICBC,0.267,100.00%
1,China Construction Bank,0.257,75.00%
2,Agricultural Bank of China,0.241,50.00%
3,Bank of America,0.192,25.00%




--- Original Dataset Updated with tech_score_model ---


,2022 Ranking,Organization Name,Industry,Country,Year Founded,CEO,Revenue (Billions),Profits (Billions),Assets (Billions),Market Value (Billions),Total Employees,tech_score_model
0,2,ICBC,Banking,China,1984,Shu Gu,208.13,54.03,5518.51,214.42,449296,0.267
1,5,China Construction Bank,Banking,China,2014,Wang Zuji,202.07,46.89,4746.95,181.32,352621,0.257
2,8,Agricultural Bank of China,Banking,China,1979,Gu Shu,181.42,37.38,4561.05,133.38,455174.0,0.241
3,9,Bank of America,Banking,United States,1998,Brian Moynihan,96.83,31.00,3238.22,303.10,208000.0,0.192


In [13]:
# ---------------------------
# 0) Install Libraries
# ---------------------------
!pip install newsapi-python pandas transformers torch --quiet

import pandas as pd
from transformers import pipeline
from newsapi import NewsApiClient
from google.colab import userdata
from tqdm.auto import tqdm

# Register tqdm for pandas so we can see progress bars
tqdm.pandas()

# ---------------------------
# 1) Load Models and API Clients
# ---------------------------
try:
    # Load the sentiment analysis model (downloads once)
    sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

    # Initialize the NewsAPI client with your key from Colab Secrets
    NEWS_API_KEY = userdata.get('NEWS_API_KEY')
    newsapi = NewsApiClient(api_key=NEWS_API_KEY)

    print("Models and API clients loaded successfully.")

except userdata.SecretNotFoundError:
    print(" ERROR: NEWS_API_KEY not found in Colab Secrets. Please add your key.")
    newsapi = None
except Exception as e:
    print(f"An error occurred during setup: {e}")

# ---------------------------
# 2) Core Analysis Function
# ---------------------------
def analyze_company_news(company_name):
    """
    Fetches news, calculates News Volume, and determines the Average Sentiment Score.
    """
    if not newsapi:
        return 0, 0.0 # Return default values if API client isn't ready

    try:
        # Fetch the top 30 most relevant English articles from the last 30 days
        all_articles = newsapi.get_everything(
            q=f'"{company_name}"', # Use quotes for exact name matching
            language='en',
            sort_by='relevancy',
            page_size=30
        )

        articles = all_articles.get('articles', [])
        if not articles:
            return 0, 0.0

        sentiments = []
        for article in articles:
            text_to_analyze = (article.get('title', "") or "") + " " + (article.get('description', "") or "")
            if text_to_analyze.strip():
                result = sentiment_analyzer(text_to_analyze)[0]
                # Convert sentiment to a number: POSITIVE=1, NEGATIVE=-1
                score = 1 if result['label'] == 'POSITIVE' else -1
                sentiments.append(score)

        # Metric 1: News Volume
        news_volume = len(articles)

        # Metric 2: Average Sentiment Score
        avg_sentiment = sum(sentiments) / len(sentiments) if sentiments else 0.0

        return news_volume, round(avg_sentiment, 3)

    except Exception as e:
        # Silently handle API errors (like hitting rate limits on the free plan)
        # print(f"NewsAPI error for {company_name}: {e}")
        return 0, 0.0

# ---------------------------
# 3) Load Data and Run Analysis
# ---------------------------
try:


    # <<< --- SET YOUR TARGET INDUSTRY HERE --- >>>
    TARGET_INDUSTRY = 'Banking'

    # Filter the DataFrame to get all companies in the selected domain
    peer_group_df = df[df['Industry'] == TARGET_INDUSTRY].copy()

    if peer_group_df.empty:
        print(f"\nERROR: Industry '{TARGET_INDUSTRY}' not found or has no companies.")
    else:
        print(f"\nAnalyzing {len(peer_group_df)} companies in the '{TARGET_INDUSTRY}' domain...")

        # Apply the analysis function to each company
        # The .progress_apply shows a helpful progress bar
        news_results = peer_group_df['Organization Name'].progress_apply(analyze_company_news)

        # Add the new metrics to the DataFrame
        peer_group_df['news_volume'] = [result[0] for result in news_results]
        peer_group_df['sentiment_score'] = [result[1] for result in news_results]

        # --- Calculate Metric 3: Share of Voice (Market Dominance) ---
        total_industry_volume = peer_group_df['news_volume'].sum()
        if total_industry_volume > 0:
            peer_group_df['share_of_voice'] = peer_group_df['news_volume'] / total_industry_volume
        else:
            peer_group_df['share_of_voice'] = 0.0

        # --- Display the final results ---
        print("\n\n--- News Analysis Complete ---")

        df_sorted = peer_group_df.sort_values(by='share_of_voice', ascending=False)

        display_cols = ['Organization Name', 'sentiment_score', 'share_of_voice']

        styled_df = df_sorted[display_cols].style.format({
            'sentiment_score': '{:+.3f}', # Show + or - for sentiment
            'share_of_voice': '{:.2%}' # Format as percentage
        }).background_gradient(cmap='Greens', subset=['share_of_voice'])

        display(styled_df)


        display_cols = ['Organization Name', 'share_of_voice']
        df=industry_df.merge(peer_group_df[ display_cols], on='Organization Name', how='left')

        print("\n\n--- Original Dataset Updated with tech_score_model ---")
        display(df.head())  # show first 10 for quick check

except FileNotFoundError:
    print("\nERROR: The file 'Forbes_2000_top_company_CLNQ11.csv' was not found. Please make sure it's uploaded.")
except Exception as e:
    print(f"\nAn error occurred: {e}")



config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


...

  0%|          | 0/4 [00:00<?, ?it/s]



--- News Analysis Complete ---


,Organization Name,sentiment_score,share_of_voice
3,Bank of America,-0.067,55.56%
0,ICBC,+0.455,20.37%
2,Agricultural Bank of China,-0.429,12.96%
1,China Construction Bank,+0.000,11.11%




--- Original Dataset Updated with tech_score_model ---


,2022 Ranking,Organization Name,Industry,Country,Year Founded,CEO,Revenue (Billions),Profits (Billions),Assets (Billions),Market Value (Billions),Total Employees,share_of_voice
0,2,ICBC,Banking,China,1984,Shu Gu,208.13,54.03,5518.51,214.42,449296,0.203704
1,5,China Construction Bank,Banking,China,2014,Wang Zuji,202.07,46.89,4746.95,181.32,352621,0.111111
2,8,Agricultural Bank of China,Banking,China,1979,Gu Shu,181.42,37.38,4561.05,133.38,455174.0,0.129630
3,9,Bank of America,Banking,United States,1998,Brian Moynihan,96.83,31.00,3238.22,303.10,208000.0,0.555556


In [14]:
!pip install pytrends -q

In [15]:
from pytrends.request import TrendReq
import time

def get_brand_interest_score(company_name):
    """
    Fetches the average Google Trends interest for a company over the last 12 months.
    """
    pytrends = TrendReq(hl='en-US', tz=360)
    try:
        pytrends.build_payload([company_name], cat=0, timeframe='today 12-m', geo='', gprop='')
        interest_df = pytrends.interest_over_time()

        if interest_df.empty or company_name not in interest_df.columns:
            return 0.0

        avg_score = interest_df[company_name].mean()
        time.sleep(1) # Add a small delay to avoid rate-limiting
        return round(avg_score, 2)
    except Exception as e:
        return 0.0

In [16]:
import pandas as pd
from tqdm.auto import tqdm
from sklearn.preprocessing import MinMaxScaler

# This assumes 'df' is your full DataFrame loaded from the CSV and
# the 'get_brand_interest_score' function is already defined.

# --- 1. NEW: Add this section to filter by industry ---
# Get and display all unique industries for you to choose from
available_industries = sorted(df['Industry'].unique())
print("--- Available Industries for Analysis ---")
for industry in available_industries:
    print(f"- {industry}")
print("-" * 40)

# <<< --- SET YOUR TARGET INDUSTRY HERE --- >>>
TARGET_INDUSTRY = 'Banking'

# Filter the DataFrame to get all companies in the selected domain
peer_group_df = df[df['Industry'] == TARGET_INDUSTRY].copy()
# ----------------------------------------------------


if peer_group_df.empty:
    print(f"\nERROR: Industry '{TARGET_INDUSTRY}' not found or has no companies.")
else:
    print(f"\nAnalyzing {len(peer_group_df)} companies in the '{TARGET_INDUSTRY}' domain...")

    # --- 2. UPDATE: Run the analysis on 'peer_group_df' instead of 'df' ---
    # Calculate brand interest scores for each company in the DataFrame
    peer_group_df['brand_interest_score'] = peer_group_df['Organization Name'].progress_apply(get_brand_interest_score)

    print("\nNormalizing scores to a 0-1 range (specific to this industry)...")

    # Initialize the Min-Max Scaler
    scaler = MinMaxScaler()

    # Apply the scaler to the score column and create a new normalized column
    peer_group_df['brand_interest_norm'] = scaler.fit_transform(peer_group_df[['brand_interest_score']])

    print("\n--- Analysis Complete ---")

    # Sort the results by the original score
    df_ranked = peer_group_df.sort_values(by='brand_interest_score', ascending=False)

    # Display the final DataFrame with both the original and normalized scores
    styled_df = df_ranked[['Organization Name', 'brand_interest_score', 'brand_interest_norm']].style.format({
        'brand_interest_score': '{:.2f}',
        'brand_interest_norm': '{:.3f}' # Show normalized score with 3 decimal places
    }).background_gradient(cmap='viridis', subset=['brand_interest_score', 'brand_interest_norm'])

    display(styled_df)


print("\n--- Original DataFrame with New Columns ---")
display(peer_group_df.head())

--- Available Industries for Analysis ---
- Banking
----------------------------------------

Analyzing 4 companies in the 'Banking' domain...


  0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
/usr/local/lib/python3.12/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(False)
/usr/local/lib/python3.12/dist-packages/pytrends/request.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('fut


Normalizing scores to a 0-1 range (specific to this industry)...

--- Analysis Complete ---


,Organization Name,brand_interest_score,brand_interest_norm
0,ICBC,80.75,1.000
2,Agricultural Bank of China,44.34,0.549
1,China Construction Bank,32.92,0.408
3,Bank of America,0.00,0.000



--- Original DataFrame with New Columns ---


,2022 Ranking,Organization Name,Industry,Country,Year Founded,CEO,Revenue (Billions),Profits (Billions),Assets (Billions),Market Value (Billions),Total Employees,share_of_voice,brand_interest_score,brand_interest_norm
0,2,ICBC,Banking,China,1984,Shu Gu,208.13,54.03,5518.51,214.42,449296,0.203704,80.75,1.000000
1,5,China Construction Bank,Banking,China,2014,Wang Zuji,202.07,46.89,4746.95,181.32,352621,0.111111,32.92,0.407678
2,8,Agricultural Bank of China,Banking,China,1979,Gu Shu,181.42,37.38,4561.05,133.38,455174.0,0.129630,44.34,0.549102
3,9,Bank of America,Banking,United States,1998,Brian Moynihan,96.83,31.00,3238.22,303.10,208000.0,0.555556,0.00,0.000000


In [17]:
# ============================================================
# MERGE ALL ANALYSIS RESULTS INTO ORIGINAL DATASET
# ============================================================

try:
    # Step 1 — Merge share_of_voice
    display_cols = ['Organization Name', 'share_of_voice']
    df_merged = industry_df.merge(peer_group_df[display_cols], on='Organization Name', how='left')

    print("\n--- Original Dataset Updated with Share of Voice ---")
    display(df_merged.head())

    # Step 2 — Merge tech_score_model
    tech_cols_to_merge = ['Organization Name', 'tech_score_model']
    df_merged = df_merged.merge(df_final[tech_cols_to_merge], on='Organization Name', how='left')

    print("\n--- Dataset Updated with tech_score_model ---")
    display(df_merged.head())

    # Step 3 — Merge PCA columns
    pca_cols_to_merge = ['Organization Name', 'Rank_in_Industry', 'PCA ScoreN']
    df_merged = df_merged.merge(final_results_df[pca_cols_to_merge], on='Organization Name', how='left')

    # Step 4 — Merge Brand Interest columns
    brand_interest_cols_to_merge = ['Organization Name', 'brand_interest_norm']
    df_merged = df_merged.merge(peer_group_df[brand_interest_cols_to_merge], on='Organization Name', how='left')

    print("\n--- Final Combined Dataset (All Columns Added) ---")
    display(df_merged)

except FileNotFoundError:
    print("\nERROR: One of the input files was not found. Please make sure all required files are uploaded.")
except Exception as e:
    print(f"\nAn error occurred: {e}")



--- Original Dataset Updated with Share of Voice ---


,2022 Ranking,Organization Name,Industry,Country,Year Founded,CEO,Revenue (Billions),Profits (Billions),Assets (Billions),Market Value (Billions),Total Employees,share_of_voice
0,2,ICBC,Banking,China,1984,Shu Gu,208.13,54.03,5518.51,214.42,449296,0.203704
1,5,China Construction Bank,Banking,China,2014,Wang Zuji,202.07,46.89,4746.95,181.32,352621,0.111111
2,8,Agricultural Bank of China,Banking,China,1979,Gu Shu,181.42,37.38,4561.05,133.38,455174.0,0.129630
3,9,Bank of America,Banking,United States,1998,Brian Moynihan,96.83,31.00,3238.22,303.10,208000.0,0.555556



--- Dataset Updated with tech_score_model ---


,2022 Ranking,Organization Name,Industry,Country,Year Founded,CEO,Revenue (Billions),Profits (Billions),Assets (Billions),Market Value (Billions),Total Employees,share_of_voice,tech_score_model
0,2,ICBC,Banking,China,1984,Shu Gu,208.13,54.03,5518.51,214.42,449296,0.203704,0.267
1,5,China Construction Bank,Banking,China,2014,Wang Zuji,202.07,46.89,4746.95,181.32,352621,0.111111,0.257
2,8,Agricultural Bank of China,Banking,China,1979,Gu Shu,181.42,37.38,4561.05,133.38,455174.0,0.129630,0.241
3,9,Bank of America,Banking,United States,1998,Brian Moynihan,96.83,31.00,3238.22,303.10,208000.0,0.555556,0.192



--- Final Combined Dataset (All Columns Added) ---


,2022 Ranking,Organization Name,Industry,Country,Year Founded,CEO,Revenue (Billions),Profits (Billions),Assets (Billions),Market Value (Billions),Total Employees,share_of_voice,tech_score_model,Rank_in_Industry,PCA ScoreN,brand_interest_norm
0,2,ICBC,Banking,China,1984,Shu Gu,208.13,54.03,5518.51,214.42,449296,0.203704,0.267,1,1.000000,1.000000
1,5,China Construction Bank,Banking,China,2014,Wang Zuji,202.07,46.89,4746.95,181.32,352621,0.111111,0.257,2,0.793350,0.407678
2,8,Agricultural Bank of China,Banking,China,1979,Gu Shu,181.42,37.38,4561.05,133.38,455174.0,0.129630,0.241,3,0.790166,0.549102
3,9,Bank of America,Banking,United States,1998,Brian Moynihan,96.83,31.00,3238.22,303.10,208000.0,0.555556,0.192,4,0.000000,0.000000


In [18]:
df_merged.columns

Index(['2022 Ranking', 'Organization Name', 'Industry', 'Country',
       'Year Founded', 'CEO', 'Revenue (Billions)', 'Profits (Billions)',
       'Assets (Billions)', 'Market Value (Billions)', 'Total Employees',
       'share_of_voice', 'tech_score_model', 'Rank_in_Industry', 'PCA ScoreN',
       'brand_interest_norm'],
      dtype='object')

In [19]:
# ===============================================================
# STEP 0 — IMPORT LIBRARIES
# ===============================================================
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# ===============================================================
# STEP 1 — DEFINE THE TOPSIS SCORING FUNCTION
# ===============================================================
def topsis_score(df, metrics, weights=None):
    """
    Calculates a TOPSIS score and normalized ranking for a given DataFrame.
    Args:
        df (pd.DataFrame): Must contain the metrics.
        metrics (list): List of metric column names to use.
        weights (list): Optional list of weights (default = equal).
    Returns:
        pd.DataFrame: DataFrame with added columns for TOPSIS Score and normalized score.
    """
    df = df.copy()

    # --- Normalize metrics to [0,1] range ---
    scaler = MinMaxScaler()
    df[metrics] = scaler.fit_transform(df[metrics])

    # --- Apply weights (equal by default) ---
    if weights is None:
        weights = np.ones(len(metrics))
    weights = np.array(weights) / np.sum(weights)

    # --- Weighted normalized decision matrix ---
    V = df[metrics].values * weights

    # --- Identify ideal best and worst solutions ---
    ideal_best = V.max(axis=0)
    ideal_worst = V.min(axis=0)

    # --- Euclidean distances to best and worst ---
    d_best = np.linalg.norm(V - ideal_best, axis=1)
    d_worst = np.linalg.norm(V - ideal_worst, axis=1)

    # --- Compute TOPSIS score (closeness coefficient) ---
    df['TOPSIS Score'] = d_worst / (d_best + d_worst + 1e-9)

    # --- Normalize final score for comparability ---
    df['TOPSIS SCOREN'] = MinMaxScaler().fit_transform(df[['TOPSIS Score']])

    return df


# ===============================================================
# STEP 2 — INDUSTRY-SPECIFIC WRAPPER FUNCTION
# ===============================================================
def run_topsis_industry_specific(df, target_industry=None, weights=None):
    """
    Runs TOPSIS scoring on an industry-specific or full dataset.
    Args:
        df (pd.DataFrame): Must contain 'Industry' and metric columns.
        target_industry (str): Optional, specify industry name to filter.
        weights (list): Optional metric weights for TOPSIS.
    Returns:
        pd.DataFrame: DataFrame with TOPSIS results and ranking.
    """

    # --- Choose metrics for analysis ---
    metrics = ['brand_interest_norm', 'share_of_voice', 'tech_score_model', 'PCA ScoreN']

    # --- Filter to target industry if specified ---
    if target_industry:
        df = df[df['Industry'] == target_industry].copy()
        print(f"Running TOPSIS for Target Industry: {target_industry}")
    else:
        print("Running TOPSIS for All Industries Combined")

    # --- Clean data (drop rows missing any metric) ---
    df = df.dropna(subset=metrics)

    if df.empty:
        print("Not enough data after cleaning for TOPSIS analysis.")
        return None

    # --- Compute TOPSIS ---
    df_scored = topsis_score(df, metrics, weights)

    # --- Rank within industry (descending order of score) ---
    df_scored['Industry Rank'] = df_scored['TOPSIS SCOREN'].rank(method='dense', ascending=False)
    df_scored = df_scored.sort_values(by='Industry Rank').reset_index(drop=True)

    # --- Return final scored table ---
    return df_scored[['Industry', 'Organization Name', 'TOPSIS SCOREN', 'Industry Rank'] + metrics]


# ===============================================================
# STEP 3 — EXAMPLE USAGE
# ===============================================================

df = pd.DataFrame(df_merged)

# Specify your target industry
TARGET_INDUSTRY = 'Banking'

# Run the analysis
final_topsis_df = run_topsis_industry_specific(df, target_industry=TARGET_INDUSTRY)

# Display results
if final_topsis_df is not None and not final_topsis_df.empty:
    print("\n--- TOPSIS Industry-Specific Ranking ---")
    display(final_topsis_df)

    # --- Identify the company with the highest TOPSIS Score Norm [0-1] ---
    best_idx = final_topsis_df['TOPSIS SCOREN'].idxmax()
    final_topsis_df['Investment Recommendation'] = ''  # create empty column
    final_topsis_df.loc[best_idx, 'Investment Recommendation'] = '★🏆 Best to invest in'

    # --- Display the final table with recommendation ---
    print("\n--- TOPSIS Results with Investment Recommendation ---")
    display(final_topsis_df)
else:
    print("\nTOPSIS analysis could not be completed. Please check the data and target industry.")

Running TOPSIS for Target Industry: Banking

--- TOPSIS Industry-Specific Ranking ---


,Industry,Organization Name,TOPSIS SCOREN,Industry Rank,brand_interest_norm,share_of_voice,tech_score_model,PCA ScoreN
0,Banking,ICBC,1.000000,1.0,1.000000,0.208333,1.000000,1.000000
1,Banking,China Construction Bank,0.451866,2.0,0.407678,0.000000,0.866667,0.793350
2,Banking,Agricultural Bank of China,0.436460,3.0,0.549102,0.041667,0.653333,0.790166
3,Banking,Bank of America,0.000000,4.0,0.000000,1.000000,0.000000,0.000000



--- TOPSIS Results with Investment Recommendation ---


,Industry,Organization Name,TOPSIS SCOREN,Industry Rank,brand_interest_norm,share_of_voice,tech_score_model,PCA ScoreN,Investment Recommendation
0,Banking,ICBC,1.000000,1.0,1.000000,0.208333,1.000000,1.000000,★🏆 Best to invest in
1,Banking,China Construction Bank,0.451866,2.0,0.407678,0.000000,0.866667,0.793350,
2,Banking,Agricultural Bank of China,0.436460,3.0,0.549102,0.041667,0.653333,0.790166,
3,Banking,Bank of America,0.000000,4.0,0.000000,1.000000,0.000000,0.000000,
